In [1]:
# read .csv into python
import pandas as pd
import numpy as np
import os
maindata = pd.read_csv('D:/Columbia/Fall 2018/PTA/Project/beeradvocate1.csv', sep='^', engine='python', 
                   header=None, error_bad_lines=False, warn_bad_lines=False)

In [59]:
maindata.columns = ['beer_name', 'beer_beerId', 'beer_brewer', 'beer_ABV', 'beer_style', 
                'review_appearance', 'review_aroma', 'review_palate', 'review_taste', 
                'review_overall', 'review_time', 'review_profileName', 'review_text']

# keep 3 columns: user name, beer name, overall score
data2 = maindata[['beer_name', 'review_profileName', 'review_overall', 'review_time']]

# n = 56855 # Number of items
# m = 33382

n = 56855 # Number of items
m = 33382
# remove NA
data2 = data2[pd.notnull(data2.beer_name)]
data2 = data2[pd.notnull(data2.review_profileName)]
data2 = data2[pd.notnull(data2.review_overall)]
print(data2.shape)

(1585696, 4)


In [60]:
# keep users with atleast 10 beers/reviews.
user = data2.review_profileName.value_counts()
# for i in range(1, 33382) :
#         if (user[i] == 9):
#             m = i 
#             break;
user_list = user.keys()[:m].tolist()

# keep top 100/56855 most reviewed beer
beer = data2.beer_name.value_counts()[:40000]
beer_list = beer.keys()[:n].tolist()

# keep (beer&user) pair in (user_list) and (beer_list)
subdata = data2[data2.beer_name.isin(beer_list)]
subdata = subdata[subdata.review_profileName.isin(user_list)]

# sort by user names
subdata = subdata.sort_values(by=['review_profileName','beer_name','review_time'])

print(subdata.shape)


(1550902, 4)


In [61]:
#subdata.groupby(['beer_name', 'review_profileName'])['review_time'].max()

data_clean = subdata.sort_values(by='review_time')

data_clean.drop_duplicates(['beer_name', 'review_profileName'], keep = 'last',
                          inplace = True)
data_clean['review_overall'] = data_clean['review_overall'].astype(float)



In [140]:
from surprise import NMF, GridSearch, Reader, Dataset, SVD, NMF
from surprise.model_selection import GridSearchCV

In [63]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_clean[['review_profileName', 'beer_name', 'review_overall']], reader)

SVD

In [64]:
from collections import defaultdict
from surprise.model_selection import train_test_split
import time
start_time = time.time()
trainset, testset = train_test_split(data, test_size=.25)
# training_set = data.build_full_trainset()
algo = SVD(n_factors = 30, lr_all = 0.01, reg_all = 0.05)
algo.fit(trainset)
#unfilled = training_set.build_anti_testset()
recommendation = algo.test(testset)

print("--- %s seconds ---" % (time.time() - start_time))

--- 38.013381481170654 seconds ---


In [100]:
user_dict = {}
for recom in recommendation:
    if recom[0] not in user_dict:
        user_dict[recom[0]] = []
    user_dict[recom[0]].append((recom[2],recom[3]))

In [112]:

rmse_dict = {}
for user in user_dict:
    err = 0
    for t in user_dict[user]:
        err = err + ((t[0] - t[1]) ** 2)
    rmse = err/ len(user_dict[user])
    beers = len(data_clean[data_clean['review_profileName'] == user]) - len(user_dict[user])
    if beers not in rmse_dict:
         rmse_dict[beers] = []
    rmse_dict[beers].append(rmse)


{96: [0.14880832784973164,
  0.4610189789570313,
  0.1771250096422176,
  0.6857638337061351,
  0.4367567811244755,
  0.18649157256940788,
  0.627739070372047,
  0.4793100412595075,
  0.23942853359821434,
  0.19653887948670978,
  0.30339967734773526,
  0.507392948429461,
  0.3796690760798738,
  0.2944474519868895,
  0.26494803591127236,
  0.2653443019101793,
  0.30146231571645465,
  1.330125688487566,
  0.37423180958959906,
  0.33114584384760204,
  0.21673118144791384,
  0.21254854561353675],
 476: [0.6595246785400914],
 765: [0.412071988157079],
 726: [0.32360609962467934, 0.44745723445704544, 0.42114312084342126],
 569: [0.2663741373724816, 0.2832426423552624, 0.29842843705871946],
 25: [0.6292529525077739,
  0.36248341973294346,
  0.20294451162372576,
  0.13574599009077462,
  0.3079691893900068,
  0.13422109381643105,
  1.676426905701736,
  0.20456153680250677,
  0.462845344170871,
  1.0630680013336722,
  0.30764395772234876,
  1.2337598824073506,
  0.9290846967068251,
  0.5914736911

In [139]:
rmse_final = {}
for beer_count in rmse_dict:
    average = sum(rmse_dict[beer_count]) / len(rmse_dict[beer_count])
    rmse_final[beer_count] =  average


In [151]:
rmse_final

{96: 0.38274672295152556,
 476: 0.6595246785400914,
 765: 0.412071988157079,
 726: 0.3974021516417154,
 569: 0.28268173892882115,
 25: 0.391525406818422,
 92: 0.2984800901901328,
 728: 0.22159095375807625,
 425: 0.5442476414859609,
 104: 0.36181006621148526,
 6: 0.597220246135826,
 671: 0.11712469514332874,
 372: 0.30513480633783574,
 106: 0.3595080038323003,
 684: 0.29976177009467,
 341: 0.3652491916491685,
 12: 0.5010914954688594,
 2305: 0.19160588247317853,
 1317: 0.4949054276510541,
 180: 0.35580171291014084,
 1599: 0.25178070621278725,
 502: 0.40995677096285194,
 738: 0.4246742371238461,
 382: 0.319551086918584,
 151: 0.3418294854651666,
 264: 0.24191153568223375,
 83: 0.3569479511850944,
 1024: 0.14507623510441822,
 699: 0.2883171041461089,
 115: 0.3917347856683276,
 33: 0.40170066951636146,
 100: 0.33080572886145676,
 930: 0.28816482816128997,
 221: 0.4099577477115365,
 78: 0.3186000839514657,
 82: 0.3876313295623312,
 155: 0.39234199787714985,
 853: 0.40674864239257297,
 2: 0.6

NMF

In [143]:

start_time = time.time()
algo = NMF(n_factors = 25, n_epochs = 50, reg_pu = 0.1, reg_qi = 0.1)
algo.fit(trainset)
nmf_recommendation = algo.test(testset)

print("--- %s seconds ---" % (time.time() - start_time))

--- 165.19624876976013 seconds ---


In [144]:
nmf_user_dict = {}
for recom in nmf_recommendation:
    if recom[0] not in nmf_user_dict:
        nmf_user_dict[recom[0]] = []
    nmf_user_dict[recom[0]].append((recom[2],recom[3]))

In [148]:
nmf_rmse_dict = {}
for user in nmf_user_dict:
    err = 0
    for t in nmf_user_dict[user]:
        err = err + ((t[0] - t[1]) ** 2)
    rmse = err/ len(nmf_user_dict[user])
    beers = len(data_clean[data_clean['review_profileName'] == user]) - len(nmf_user_dict[user])
    if beers not in nmf_rmse_dict:
         nmf_rmse_dict[beers] = []
    nmf_rmse_dict[beers].append(rmse)

In [149]:
nmf_rmse_final = {}
for beer_count in nmf_rmse_dict:
    average = sum(nmf_rmse_dict[beer_count]) / len(nmf_rmse_dict[beer_count])
    nmf_rmse_final[beer_count] =  average


In [150]:
nmf_rmse_final

{96: 0.40295923536590367,
 476: 0.6607694916494846,
 765: 0.43521266341729503,
 726: 0.4097658982549377,
 569: 0.30590761414731926,
 25: 0.39419202402346293,
 92: 0.3028072647735558,
 728: 0.2172713119609061,
 425: 0.5382222809978903,
 104: 0.36968143245690277,
 6: 0.6574386218451997,
 671: 0.1399098864960079,
 372: 0.3156971878202839,
 106: 0.36684777972289523,
 684: 0.3108262488157607,
 341: 0.3880449743145987,
 12: 0.5433587989517343,
 2305: 0.2106355236429342,
 1317: 0.48935195317866387,
 180: 0.3661969801535667,
 1599: 0.2504731259448157,
 502: 0.40006602300116273,
 738: 0.42067999041640003,
 382: 0.32221836788136715,
 151: 0.3544792985608912,
 264: 0.24483252906091302,
 83: 0.3850451079752449,
 1024: 0.17736601459876603,
 699: 0.29680892511555235,
 115: 0.3992640893493979,
 33: 0.43019857649554405,
 100: 0.3391460739558595,
 930: 0.2909204028807072,
 221: 0.411204583289852,
 78: 0.3259254889532555,
 82: 0.4075736121979896,
 155: 0.387968299178463,
 853: 0.4244987190856501,
 2: 0.

In [152]:
import json
with open('nmf_rmse.json', 'w') as outfile:
    json.dump(rmse_dict, outfile)